In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.cluster import KMeansClusterer, cosine_distance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

df = pd.read_excel('Stack_Overflow_Questions_Clean_EDA_Data.xlsx')
df5 = df[df['Answer Count']>0]
text1=list(df5['QDescription'])
len(df5)

tf_vectorizer = CountVectorizer(min_df=5, stop_words='english')
tf = tf_vectorizer.fit_transform(text1)
tf_feature_names = tf_vectorizer.get_feature_names()

num_topics = 10
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='batch',\
                                max_iter=50,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(tf)


iteration: 1 of max_iter: 50, perplexity: 1384.6069
iteration: 2 of max_iter: 50, perplexity: 1245.5682
iteration: 3 of max_iter: 50, perplexity: 1156.2105
iteration: 4 of max_iter: 50, perplexity: 1094.6956
iteration: 5 of max_iter: 50, perplexity: 1049.4683
iteration: 6 of max_iter: 50, perplexity: 1015.9801
iteration: 7 of max_iter: 50, perplexity: 993.2073
iteration: 8 of max_iter: 50, perplexity: 975.9708
iteration: 9 of max_iter: 50, perplexity: 963.4734
iteration: 10 of max_iter: 50, perplexity: 953.5496
iteration: 11 of max_iter: 50, perplexity: 946.3539
iteration: 12 of max_iter: 50, perplexity: 940.9961
iteration: 13 of max_iter: 50, perplexity: 936.3878
iteration: 14 of max_iter: 50, perplexity: 932.7422
iteration: 15 of max_iter: 50, perplexity: 929.4325
iteration: 16 of max_iter: 50, perplexity: 926.5646
iteration: 17 of max_iter: 50, perplexity: 923.9936
iteration: 18 of max_iter: 50, perplexity: 921.9823
iteration: 19 of max_iter: 50, perplexity: 919.9590
iteration: 20 o

In [4]:
df1.drop('actual_class',axis=1,inplace=True)

In [5]:
df6=pd.DataFrame()
result=pd.concat([df5,df1],axis=1,join_axes=[df5.index])
result.head()

,Question Id,Votes,Answer Count,Views,Question,QDescription,User,Reputation Score,Gold Badge Count,Silver Badge Count,Bronze Badge Count,Tags,QDescription length,cluster
13,49369729,1,1,14,Challenges with ReadTheDocs,I've been tasked with hosting our Python API d...,Robert_LY,75,0,0,8,python documentation read-the-docs,203,0.0
16,49369657,1,1,19,From Python to R - DataFrame from string,I have the following working example in Python...,Jan,20700,5,5,5,python r regex,204,6.0
23,49369594,0,1,15,n-depth tree: set parent value based on childr...,In a n-depth dict where values are set in the ...,TMichel,1472,3,3,3,python,204,3.0
25,49369548,-2,3,34,Invalid syntax Sum += 1,It says I have invalid syntax at Sum += 1. If ...,jennie saldana,1,0,0,3,python list int,203,5.0
27,49369523,0,1,16,OSError: [Errno 22] Invalid argument: (Read fi...,I have a geojson file stored on my laptop and ...,otteheng,271,0,2,2,python,204,0.0


In [7]:
import gensim
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

def cluster_similarity(new_question,cluster,value):
#Convert documents into list of tokens
    
    ques_tokens=[[w.lower() for w in word_tokenize(text)]
                for text in cluster]

# Create a dictionary and this dictionary maps every word to a number

    diction=gensim.corpora.Dictionary(ques_tokens)

# Create a bag of words. It lists the number of times each word occurs in the document.
    
    corpus=[diction.doc2bow(ques_t) for ques_t in ques_tokens]

# Create a tf - idf model

    tf_idf=gensim.models.TfidfModel(corpus)
    
# Create a similarity measure object

    sim=gensim.similarities.Similarity('C:/Users/yash1/Documents/Web Analytics/Project',tf_idf[corpus],num_features=len(diction))

# Create a query document, that is, the new question and convert it to tf - idf 
    
    query=[w.lower() for w in word_tokenize(new_question)]

    query_bow=diction.doc2bow(query)

    query_tf_idf=tf_idf[query_bow]

# The output is an array of document similarities to the new question
    
    sim[query_tf_idf]
# Store the similarity values in a dataframe with index

    s=(list(enumerate(sim[query_tf_idf])))
    sim_df = pd.DataFrame(s,columns=['Index','Similarity Values'])

# If the value is 1, it returns mean similarity values for our 10 clusters 
# Else it returns the top 10 questions similar to the new question asked in the cluster with maximum mean similarity value   
    if value==1:
        return(np.mean(sim_df['Similarity Values']))
    else:
        sort_by_life = sim_df.sort_values('Similarity Values',ascending=False)
        return(sort_by_life.head(n=10))

C:\Users\yash1\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
similarity = []
x=1
for i in range(10):
    clus = result.loc[result['cluster'] == i]
    ques = clus['Question']
    new_quest="Specific type of webscraping [on hold]"
    a = cluster_similarity(new_quest,ques,x)
    similarity.append(a) 
print(similarity)

[0.02365704809214167, 0.02273894798279715, 0.024369553824805695, 0.018350023480873275, 0.04019069545628393, 0.027863872698812842, 0.035337866765671765, 0.025909023763576768, 0.050765908090397716, 0.024345085332260548]


In [9]:
# Print the maximum mean similarity value for the new questiion asked.
# The index corresponds to the cluster number for the maximum mean similarity value

import operator
index, value = max(enumerate(similarity), key=operator.itemgetter(1))
print(index)
print(value)

8
0.050765908090397716


In [10]:
# This block returns the similarity values for the top 10 questions similar to the new question asked.
# We get the cluster value from the previous block.

y=0
clust=result.loc[result['cluster']==index]
q=clust['Question']
#print(q)
new_q="Specific type of webscraping [on hold]"
k=cluster_similarity(new_q,q,y)
print(k)

     Index  Similarity Values
63      63           0.470746
101    101           0.416636
70      70           0.412094
87      87           0.402996
43      43           0.375636
34      34           0.331571
83      83           0.308178
85      85           0.286638
30      30           0.273958
40      40           0.241262


In [11]:
# The top 10 questions that are similar to the new question asked. 

idx=pd.DataFrame(result.loc[result['cluster']==index])
idx.head()
ids=[]
i=0
ids=k["Index"]
ids=list(ids)
#print(ids[0])
a=pd.DataFrame()
while i<10:
    a=a.append(idx.iloc[[ids[i]]],ignore_index=True)
    i=i+1
a

,Question Id,Votes,Answer Count,Views,Question,QDescription,User,Reputation Score,Gold Badge Count,Silver Badge Count,Bronze Badge Count,Tags,QDescription length,cluster
0,49303054,-3,1,21,Retrieve all the values of one column of Datas...,class Publisher(models.Model):\n name = mod...,RISHABH BANSAL,1,0,0,6,python django django-models,204,8.0
1,49278902,-1,1,43,optimize nested loop in python [on hold],I am writing 2-opt algorithm and want to optim...,Tomonaga,4,0,0,1,python nested-loops,204,8.0
2,49297302,-4,2,33,Tkinter progress bar [on hold],I looked through the internet and couldn't rea...,Ben,3,0,0,2,python tkinter python-multithreading,203,8.0
3,49287715,-1,1,29,How to write multiple arrays of different dime...,"I have a header array 1x18, and 6 arrays of da...",user9493986,1,0,0,0,python arrays python-2.7 csv export-to-csv,204,8.0
4,49336667,-3,2,42,Python calling a function in an if statement [...,"I've defined the functions mathsquiznormal, et...",Bojangles,1,0,0,0,python,198,8.0
5,49344464,0,1,32,ansible win_psexec and python virtual environm...,Long story short. I have a gui interactive py...,Rick Baker,321,0,2,2,python windows ansible,197,8.0
6,49288177,-1,1,26,Python to pack and split folder with its conte...,I've recently started to learn Python and at t...,Tomasz Tarnowski,189,1,1,1,python zip 7zip packing,202,8.0
7,49287997,0,2,51,What Have I Done Wrong? | Python Code Not Runn...,Why python code dose not seem to want to run a...,Steelingsword94,2,0,0,1,python,204,8.0
8,49346981,-5,1,40,Convert binary files(malwares) into RGB images...,I'm working on a malware classification proble...,hardik0,7,0,0,3,python numpy machine-learning deep-learning co...,204,8.0
9,49338413,-4,1,42,Can we put in ascending order a list (input ()...,"I made an exercise with France IOI, but i woul...",Yacine Alloul,1,0,0,0,python python-3.x,188,8.0


In [12]:
# Example

idx=pd.DataFrame(result.loc[result['cluster']==index])
idx.iloc[63]

Question Id                                                     49303054
Votes                                                                 -3
Answer Count                                                           1
Views                                                                 21
Question               Retrieve all the values of one column of Datas...
QDescription           class Publisher(models.Model):\n    name = mod...
User                                                      RISHABH BANSAL
Reputation Score                                                       1
Gold Badge Count                                                       0
Silver Badge Count                                                     0
Bronze Badge Count                                                     6
Tags                                         python django django-models
QDescription length                                                  204
cluster                                            